<a href="https://colab.research.google.com/github/jeong-eunjeong/MasterThesis/blob/main/1_DART_api_%EC%82%AC%EC%9B%90_%EC%88%98_%EB%B0%8F_%EC%82%AC%EC%97%85%EC%9E%90%EB%B2%88%ED%98%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#★DART Open API 연결 방법

**기본적으로 https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS001 참고**\
[연결 시 필요사항]
1. 기업별 고유번호\
: "2.DART 고유번호 불러오기" 코드와 내 데이터 매칭하여 고유번호 찾기
2. URL에서 corp_code에 고유번호 연결\
: opendart 페이지에 접속하여 제공하는 URL을 기반으로 원하는 항목 찾기

#★APIkey

In [ ]:
APIkey = 'DART에서_부여받은_APIkey'

#1.DART 고유번호 불러오기

In [ ]:
import pandas as pd
import requests
import xml.etree.ElementTree as et
from io import BytesIO
from zipfile import ZipFile
import urllib
import json

### OPENDART 개발 가이드
# https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS001

# OpenDART에서 Zipfile 받아와 객체에 저장하기
u = requests.get('https://opendart.fss.or.kr/api/corpCode.xml', params={'crtfc_key':APIkey})
zipfile_bytes = u.content                                                            # 요청한 파일의 바이트 데이터를 저장합니다
zipfile_obj = ZipFile(BytesIO(zipfile_bytes))                                        # 바이트 데이터로부터 ZipFile 객체를 생성합니다.

# 압축을 풀어서 XML File을 string으로 담기
xmlfile_objs = {name: zipfile_obj.read(name) for name in zipfile_obj.namelist()}     # 압축 파일 내부의 모든 파일을 읽어서 딕셔너리로 저장합니다.
xml_str = xmlfile_objs['CORPCODE.xml'].decode('utf-8')                               # 'CORPCODE.xml' 파일의 내용을 utf-8 인코딩으로 디코딩하여 문자열로 저장합니다.

# XML String을 가져와서 DataFrame에 담기
xroot = et.fromstring(xml_str)

df_cols = ["corp_code", "corp_name", "stock_code", "modify_date"]
rows = []

for node in xroot:
    res = []
    for el in df_cols[0:]:
        if node is not None and node.find(el) is not None:
            res.append(node.find(el).text)
        else:
            res.append(None)
    rows.append({df_cols[i]: res[i]
                  for i, _ in enumerate(df_cols)})

dart = pd.DataFrame(rows, columns=df_cols)

# DataFrame 검색
dart[(dart['corp_name'] == '삼성전자')].iloc[0].corp_code

'00126380'

In [ ]:
dart.head()

,corp_code,corp_name,stock_code,modify_date
0,00434003,다코,,20170630
1,00434456,일산약품,,20170630
2,00430964,굿앤엘에스,,20170630
3,00432403,한라판지,,20170630
4,00388953,크레디피아제이십오차유동화전문회사,,20170630


In [ ]:
df_dart = dart.sort_values(by=['corp_name','modify_date'], ascending=False).drop_duplicates(['corp_name']).reset_index()

#2.corp_code 붙이기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
raw=pd.read_excel('/content/drive/MyDrive/★2023_석사논문/데이터/2023년도ESG등급.xlsx', sheet_name='3차')

In [ ]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321 entries, 0 to 320
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   NO                      321 non-null    int64  
 1   기업명                     321 non-null    object 
 2   TS기업명                   321 non-null    object 
 3   종목코드                    321 non-null    int64  
 4   ESG등급                   321 non-null    object 
 5   환경                      321 non-null    object 
 6   사회                      321 non-null    object 
 7   지배구조                    321 non-null    object 
 8   평가년도                    321 non-null    int64  
 9   자산총계                    321 non-null    int64  
 10  부채총계                    321 non-null    int64  
 11  평균시가총액                  321 non-null    int64  
 12  Tobin's_Q               320 non-null    float64
 13  합계_1인년간평균급여(천원)         321 non-null    int64  
 14  광고선전비(천원)               321 non-null    in

In [ ]:
df=raw
df['corp_name']=df['기업명']
df

,NO,기업명,TS기업명,종목코드,ESG등급,환경,사회,지배구조,평가년도,자산총계,...,지출총액총액(천원),매출액(요약)(백만원),영업이익(요약)(백만원),잡플래닛_별점,잡플래닛_승진기회 및 가능성,잡플래닛_복지 및 급여,잡플래닛_업무와 삶의 균형,잡플래닛_사내문화,잡플래닛_경영진,corp_name
0,492,아난티,아난티,25980,C,B+,C,D,2023,670903601000,...,0,161556,69253,2.200000,2.520000,2.240000,2.840000,2.400000,1.960000,아난티
1,491,아남전자,아남전자,8700,D,D,C,C,2023,150922633000,...,6310000,457296,13985,2.125000,2.000000,1.875000,2.187500,2.000000,1.687500,아남전자
2,468,애경산업,애경산업,18250,A,A,A+,A,2023,477316913000,...,15166000,589759,37546,2.904110,2.657534,2.534247,3.150685,2.821918,2.136986,애경산업
3,467,애경케미칼,애경케미칼,161000,A,A,A,B+,2023,1062980988000,...,13571183,1712780,90524,2.380000,2.440000,2.240000,2.840000,2.080000,1.660000,애경케미칼
4,466,에넥스,에넥스,11090,C,C,C,C,2023,126885952000,...,219000,202907,-23350,2.026316,2.078947,1.973684,2.657895,1.894737,1.605263,에넥스
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316,7,휴비스,휴비스,79980,A,B+,A+,B+,2023,707458895000,...,7553000,829338,-71559,2.566038,2.622642,2.924528,2.773585,2.509434,1.830189,휴비스
317,6,휴스틸,휴스틸,5010,C,C,C,C,2023,984915470000,...,606109,764919,111707,2.500000,2.000000,2.444444,3.277778,2.611111,1.888889,휴스틸
318,5,휴온스,휴온스,243070,B,C,B+,B+,2023,461646637000,...,35683845,455057,39062,2.615385,3.000000,3.000000,2.923077,2.400000,2.246154,휴온스
319,4,휴젤,휴젤,145020,B,C,B+,B,2023,826560147000,...,16641929,223135,70167,2.553191,2.542553,2.446809,3.095745,2.840426,2.180851,휴젤


In [ ]:
data = pd.merge(df_dart,df,on='corp_name')

In [ ]:
data

,index,corp_code,corp_name,stock_code,modify_date,NO,기업명,TS기업명,종목코드,ESG등급,...,(계속사업손익)법인세비용(요약)(백만원),지출총액총액(천원),매출액(요약)(백만원),영업이익(요약)(백만원),잡플래닛_별점,잡플래닛_승진기회 및 가능성,잡플래닛_복지 및 급여,잡플래닛_업무와 삶의 균형,잡플래닛_사내문화,잡플래닛_경영진
0,100147,00167208,흥아해운,003280,20231010,1,흥아해운,흥아해운,3280,B,...,-544,0,146030,25067,2.333333,2.444444,2.333333,1.888889,2.000000,1.777778
1,101222,00888347,휴젤,145020,20230915,4,휴젤,휴젤,145020,B,...,7666,16641929,223135,70167,2.553191,2.542553,2.446809,3.095745,2.840426,2.180851
2,72938,01159853,휴온스,243070,20221007,5,휴온스,휴온스,243070,B,...,17546,35683845,455057,39062,2.615385,3.000000,3.000000,2.923077,2.400000,2.246154
3,86847,00156488,휴스틸,005010,20230223,6,휴스틸,휴스틸,5010,C,...,28210,606109,764919,111707,2.500000,2.000000,2.444444,3.277778,2.611111,1.888889
4,93054,01554508,휴비스,,20230227,7,휴비스,휴비스,79980,A,...,-17260,7553000,829338,-71559,2.566038,2.622642,2.924528,2.773585,2.509434,1.830189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,89662,00139764,에넥스,011090,20221130,466,에넥스,에넥스,11090,C,...,535,219000,202907,-23350,2.026316,2.078947,1.973684,2.657895,1.894737,1.605263
316,95766,00936787,애경케미칼,161000,20230913,467,애경케미칼,애경케미칼,161000,A,...,32088,13571183,1712780,90524,2.380000,2.440000,2.240000,2.840000,2.080000,1.660000
317,68601,00139454,애경산업,018250,20220516,468,애경산업,애경산업,18250,A,...,6073,15166000,589759,37546,2.904110,2.657534,2.534247,3.150685,2.821918,2.136986
318,62783,00138516,아남전자,008700,20220225,491,아남전자,아남전자,8700,D,...,2399,6310000,457296,13985,2.125000,2.000000,1.875000,2.187500,2.000000,1.687500


In [ ]:
data.to_excel(excel_writer='corpcode_ESG2023.xlsx')

#3.사원수 구하기

In [ ]:
import urllib
import json

In [ ]:
URL = 'https://opendart.fss.or.kr/api/empSttus.json?crtfc_key={APIKEY}&corp_code=00365387&bsns_year=2022&reprt_code=11011'.format(APIKEY = APIkey)
urllib.request.urlopen(URL).read().decode("utf-8")

'{"status":"000","message":"정상","list":[{"rcept_no":"20230321001546","corp_cls":"Y","corp_code":"00365387","corp_name":"AJ네트웍스","sexdstn":"남","fo_bbm":"렌탈부문","reform_bfe_emp_co_rgllbr":"-","reform_bfe_emp_co_cnttk":"-","reform_bfe_emp_co_etc":"-","rgllbr_co":"191","rgllbr_abacpt_labrr_co":"5","cnttk_co":"4","cnttk_abacpt_labrr_co":"0","sm":"195","avrg_cnwk_sdytrn":"5.4","fyer_salary_totamt":"11,480,277,000","jan_salary_am":"58,873,000","rm":"-"},{"rcept_no":"20230321001546","corp_cls":"Y","corp_code":"00365387","corp_name":"AJ네트웍스","sexdstn":"여","fo_bbm":"렌탈부문","reform_bfe_emp_co_rgllbr":"-","reform_bfe_emp_co_cnttk":"-","reform_bfe_emp_co_etc":"-","rgllbr_co":"89","rgllbr_abacpt_labrr_co":"1","cnttk_co":"2","cnttk_abacpt_labrr_co":"0","sm":"91","avrg_cnwk_sdytrn":"4.2","fyer_salary_totamt":"3,585,003,000","jan_salary_am":"39,396,000","rm":"-"},{"rcept_no":"20230321001546","corp_cls":"Y","corp_code":"00365387","corp_name":"AJ네트웍스","sexdstn":"남","fo_bbm":"파렛트부문","reform_bfe_emp_co_rgllb

In [ ]:
u1 = urllib.request.urlopen(URL).read().decode("utf-8")
#str타입을 dictionary로 읽어옴
import json
J = json.loads(u1)
J['list']

[{'rcept_no': '20230321001546',
  'corp_cls': 'Y',
  'corp_code': '00365387',
  'corp_name': 'AJ네트웍스',
  'sexdstn': '남',
  'fo_bbm': '렌탈부문',
  'reform_bfe_emp_co_rgllbr': '-',
  'reform_bfe_emp_co_cnttk': '-',
  'reform_bfe_emp_co_etc': '-',
  'rgllbr_co': '191',
  'rgllbr_abacpt_labrr_co': '5',
  'cnttk_co': '4',
  'cnttk_abacpt_labrr_co': '0',
  'sm': '195',
  'avrg_cnwk_sdytrn': '5.4',
  'fyer_salary_totamt': '11,480,277,000',
  'jan_salary_am': '58,873,000',
  'rm': '-'},
 {'rcept_no': '20230321001546',
  'corp_cls': 'Y',
  'corp_code': '00365387',
  'corp_name': 'AJ네트웍스',
  'sexdstn': '여',
  'fo_bbm': '렌탈부문',
  'reform_bfe_emp_co_rgllbr': '-',
  'reform_bfe_emp_co_cnttk': '-',
  'reform_bfe_emp_co_etc': '-',
  'rgllbr_co': '89',
  'rgllbr_abacpt_labrr_co': '1',
  'cnttk_co': '2',
  'cnttk_abacpt_labrr_co': '0',
  'sm': '91',
  'avrg_cnwk_sdytrn': '4.2',
  'fyer_salary_totamt': '3,585,003,000',
  'jan_salary_am': '39,396,000',
  'rm': '-'},
 {'rcept_no': '20230321001546',
  'corp_c

In [ ]:
row = []

for i in range(len(data)):
  try:
    row.append(json.loads(urllib.request.urlopen('https://opendart.fss.or.kr/api/empSttus.json?crtfc_key={APIKEY}&corp_code={CODE}&bsns_year=2022&reprt_code=11011'.format(APIKEY = APIkey, CODE = data['corp_code'][i])).read().decode("utf-8"))['list'])
  except:
    print(data['corp_name'][i])

row2 = sum(row, [])
people2023 = pd.DataFrame(row2)

people2023.to_excel(excel_writer='사원수_ESG2023.xlsx')

휴비스
화신
한성기업
피에스케이
티에스이
태양금속
코아스
케이씨텍
조흥
조선내화
일성건설
이화산업
우성
에이엔피
에스에프에이


In [ ]:
people2023

,rcept_no,corp_cls,corp_code,corp_name,sexdstn,fo_bbm,reform_bfe_emp_co_rgllbr,reform_bfe_emp_co_cnttk,reform_bfe_emp_co_etc,rgllbr_co,rgllbr_abacpt_labrr_co,cnttk_co,cnttk_abacpt_labrr_co,sm,avrg_cnwk_sdytrn,fyer_salary_totamt,jan_salary_am,rm
0,20230320001057,Y,00167208,흥아해운,남,육상근무,-,-,-,19,-,-,1,20,8년,"1,532,425,000","76,621,000",-
1,20230320001057,Y,00167208,흥아해운,여,육상근무,-,-,-,7,-,-,1,8,4년 2개월,"444,709,000","55,589,000",-
2,20230320001057,Y,00167208,흥아해운,남,해상근무,-,-,-,129,-,14,-,143,4년 2개월,"12,459,677,000","87,131,000",-
3,20230320001057,Y,00167208,흥아해운,여,해상근무,-,-,-,-,-,-,-,-,-,-,-,-
4,20230320001057,Y,00167208,흥아해운,남,외국인선원,-,-,-,-,-,176,-,176,11개월,"4,045,341,000","22,985,000",-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050,20230321000733,K,00161781,아난티,여,리조트운영,-,-,-,159,-,1,-,160,2년 3개월,"5,041,751,000","28,007,000",-
1051,20230321000733,K,00161781,아난티,남,공사부문,-,-,-,15,-,2,-,17,3년 3개월,"1,193,062,000","86,662,000",-
1052,20230321000733,K,00161781,아난티,여,공사부문,-,-,-,1,-,-,-,1,7개월,"40,804,000","40,804,000",-
1053,20230321000733,K,00161781,아난티,남,본사관리,-,-,-,73,-,8,-,81,2년,"7,573,759,000","92,856,000",-


#4.사업자등록번호 연결

In [ ]:
corp_name=[]
corp_code=[]
bizr_no=[]

for i in range(len(data)):
  try:
    corp_name.append(json.loads(urllib.request.urlopen('https://opendart.fss.or.kr/api/company.json?crtfc_key={APIKEY}&corp_code={CODE}'.format(APIKEY = APIkey, CODE = data['corp_code'][i])).read().decode("utf-8"))["corp_name"])
    corp_code.append(json.loads(urllib.request.urlopen('https://opendart.fss.or.kr/api/company.json?crtfc_key={APIKEY}&corp_code={CODE}'.format(APIKEY = APIkey, CODE = data['corp_code'][i])).read().decode("utf-8"))["corp_code"])
    bizr_no.append(json.loads(urllib.request.urlopen('https://opendart.fss.or.kr/api/company.json?crtfc_key={APIKEY}&corp_code={CODE}'.format(APIKEY = APIkey, CODE = data['corp_code'][i])).read().decode("utf-8"))["bizr_no"])
  except:
    print(i,data['기업명'][i])

df_biz=pd.DataFrame({"corp_name":corp_name,
                     "corp_code":corp_code,
                     "bizr_no":bizr_no})

df_biz

,corp_name,corp_code,bizr_no
0,흥아해운(주),00167208,1208162522
1,휴젤(주),00888347,2118704515
2,(주)휴온스,01159853,8638700270
3,(주)휴스틸,00156488,1078111563
4,(주)휴비스,01554508,3148610420
...,...,...,...
315,(주)에넥스,00139764,2208131765
316,애경케미칼(주),00936787,1138663305
317,애경산업(주),00139454,1138111573
318,아남전자(주),00138516,1228102613


In [ ]:
print(len(corp_name))
print(len(corp_code))
print(len(bizr_no))

264
263
263


In [ ]:
corp_name.remove('(주)대현')
print(len(corp_name))

263


In [ ]:
df_biz=pd.DataFrame({"corp_name":corp_name,
                     "corp_code":corp_code,
                     "bizr_no":bizr_no})